# Azure Container Apps' Dynamic Sessions with Autogen

Demonstrates how to use Dynamic Sessions feature of Azure Container Apps to execute code

In [1]:
! pip -qqq install pyautogen matplotlib yfinance python-dotenv azure-identity

In [1]:
import os
from typing import List

from autogen import ConversableAgent, config_list_from_json
from autogen.coding import CodeBlock, CodeExecutor, CodeExtractor, CodeResult, MarkdownCodeExtractor

import requests
from typing import List

### Setup Sessions pool and get Sessions endpoint

Azure Container Apps is a serverless platform that enables you to run containerized workloads without managing the underlying infrastructure. Dynamic sessions adds the ability to execute untrusted code in secure, sandboxed environments at scale. [Learn more here](https://techcommunity.microsoft.com/t5/apps-on-azure-blog/new-secure-sandboxes-at-scale-with-azure-container-apps-dynamic/ba-p/4142148). 

![sessions](https://techcommunity.microsoft.com/t5/image/serverpage/image-id/581694i61DE419D1A04B7F5/image-size/large?v=v2&px=999)

## Prerequisites

1. Get the Pool endpoint from [Azure Container Apps - Sessions Code Interpreter](https://learn.microsoft.com/en-us/azure/container-apps/sessions-code-interpreter?tabs=azure-cli).

2. Copy `.env.example` to `.env` and update it with your sessions endpoint.

3. Copy `OAI_CONFIG_LIST.example` to `OAI_CONFIG_LIST` and update it with your Azure OpenAI values.

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

ACA_SESSIONS_ENDPOINT = os.getenv('ACA_SESSIONS_ENDPOINT')

#### Why Managed Identity:
Managed Identity (MI) simplifies Azure service authentication by automatically managing credentials. Using MI, there is no need to store or manage secrets in your application, significantly reducing the risk of credential leakage and enhancing security.

#### How `DefaultAzureCredential` Works:
`DefaultAzureCredential` attempts to find a best-available credential from a series of providers, including managed identity and developer credentials from Azure CLI or Visual Studio. This makes it both flexible and convenient for development and production environments without code changes.

![Azure Identity](https://raw.githubusercontent.com/Azure/azure-sdk-for-net/main/sdk/identity/Azure.Identity/images/mermaidjs/DefaultAzureCredentialAuthFlow.svg)

#### Setup for Azure OpenAI:
1. **Azure Managed Identity**:
   - Ensure that the Azure resource (e.g., Azure OpenAI, Logic Apps, AI Search) running in this notebook is assigned a Managed Identity.
   - Configure the Managed Identity to have access to the Azure OpenAI service by setting appropriate role assignments in Azure RBAC.

- **Keys required in .env File**:
  - `OPENAI_ENDPOINT`: The endpoint URL for the Azure OpenAI service.
  - `OPENAI_MODEL_NAME`: The deployed model name in Azure OpenAI


## ACA code executor class implemented by subclassing the `CodeExecutor` protocol and implementing the `execute_code_blocks` method.

In [4]:
import requests
from typing import List
from azure.identity import DefaultAzureCredential
from autogen.coding import CodeBlock, CodeExecutor, CodeExtractor, CodeResult, MarkdownCodeExtractor

class ACASessionsExecutor(CodeExecutor):

    @property
    def code_extractor(self) -> CodeExtractor:
        # Extract code from markdown blocks.
        return MarkdownCodeExtractor()

    def __init__(self, pool_management_endpoint: str) -> None:
        self.pool_management_endpoint = pool_management_endpoint
        self.access_token = None

    def ensure_access_token(self) -> None:
        """Ensures that the access token is valid and updates it if necessary."""
        if not self.access_token:
            credential = DefaultAzureCredential()
            scope = "https://dynamicsessions.io" # "https://acasessions.io/.default" could also be used
            self.access_token = credential.get_token(scope).token

    def execute_code_blocks(self, code_blocks: List[CodeBlock]) -> CodeResult:
        self.ensure_access_token()  # Ensure the access token is valid before making calls
        log = ""
        print(self.access_token)
        
        exitcode = 0
        headers = {
            "Authorization": f"Bearer {self.access_token}"
        }

        for code_block in code_blocks:
            properties = {
                "identifier": "adslfjlad",
                "codeInputType": "inline",
                "executionType": "synchronous",
                "pythonCode": code_block.code,
                "timeoutInSeconds": 100
            }

            try:
                response = requests.post(
                    self.pool_management_endpoint + "/python/execute",
                    headers=headers,
                    json={"properties": properties}
                )
                response.raise_for_status()  # Raises an HTTPError for bad responses
                data = response.json()
                log += data.get("stdout", "") + data.get("stderr", "")
                if "result" in data and data["result"] is not None:
                    log += str(data["result"])
                if "error" in data:
                    log += f"\n{data['error']}"
                    exitcode = 1
            except requests.RequestException as e:
                log += f"\nError while sending code block to endpoint: {str(e)}"
                exitcode = 1
                break

        return CodeResult(exit_code=exitcode, output=log)


ACA Code executor used in Agents

In [5]:
config_list = config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4"],
    },
)

code_writer_agent = ConversableAgent(
    name="CodeWriter",
    system_message="You are a helpful AI assistant.\n"
    "You use your coding skill to solve problems.\n"
    "You have access to a IPython kernel to execute Python code.\n"
    "You output only valid python code. \n"
    "This valid code will be executed in a sandbox, resutling in result, stdout, or stderr. \n"
    "All necessary libraries have already been installed.\n"
    "Once the task is done, returns 'TERMINATE'.",
    llm_config={"config_list":config_list},
)

aca_pool_management_endpoint = ACA_SESSIONS_ENDPOINT
aca_sessions_executor = ACASessionsExecutor(aca_pool_management_endpoint)

code_executor_agent = ConversableAgent(
    name="CodeExecutor",
    llm_config=False,
    code_execution_config={"executor": aca_sessions_executor},
    is_termination_msg=lambda msg: "TERMINATE" in msg.get("content", "").strip().upper(),
)

Simple math task for testing ACA Sessions

In [6]:
chat_result = code_executor_agent.initiate_chat(
    code_writer_agent,
    message="What is 30 plus 20?",
)

CodeExecutor (to CodeWriter):

What is 30 plus 20?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
CodeWriter (to CodeExecutor):

To solve this, you can simply add the two numbers in Python:

```python
result = 30 + 20
print(result)
```

When you run this code, it will output the sum of 30 and 20. 

Would you like me to execute this code to get the result?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
eyJ0eXAiOiJKV1QiLCJyaCI6IjAuQW84RF9YSmZkVHdadDBDMVo3YkJMQXplY3pfWGZTd2hlbHRJdVgyaVVJLWhVc1AtQU5NLiIsImFsZyI6IlJTMjU2IiwieDV0IjoiTDFLZktGSV9qblhid1djMjJ4Wnh3MXNVSEgwIiwia2lkIjoiTDFLZktGSV9qblhid1djMjJ4Wnh3MXNVSEgwIn0.eyJhdWQiOiJodHRwczovL2R5bmFtaWNzZXNzaW9ucy5pbyIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0Lzc1NWY3MmZkLTE5M2MtNDBiNy1iNTY3LWI2YzEyYzBjZGU3My8iLCJpYXQiOjE3MTQ5MjM1MTUsIm5iZiI6MTcxNDkyMzUxN

C:\Users\gok\AppData\Roaming\Python\Python312\site-packages\autogen\agentchat\conversable_agent.py:1306: UserWarning: Extracted_response from {response} is None.
  warnings.warn("Extracted_response from {response} is None.", UserWarning)
